实现读取本地agrovc 的rdf文件，构建核心的知识库，以成为其他本体链接的对象，在此基础上，构建api实现与业务系统的融合。

1. 读取本地agrovc rdf 文件
   * 简单直接做法是通过rdflib读取，但是由于文件过大，不可能在生产环境中使用这种方式。在实际应用中必须连接本地grpahdb数据库，或者读取远端sqarqlpoint
2. 读取本地plant-ontology 文件，或C3PO文件
3. 构建api实现提接口，通过接口读取本地的rdf文件
4. 基于rdflib，对plan本体进行基本的tbox统计描述

In [25]:
import rdflib
from rdflib import Graph

from rdflib import Graph, URIRef, RDF, RDFS,OWL
 
 
 
def edaRdf(g:Graph):
    eda={}
   
 
    # 统计类的数量
    # 这里同时考虑了OWL和RDFS定义的类
    classes = set(g.subjects(RDF.type, OWL.Class)) 
    num_classes = len(classes)
    

    # 统计属性的数量
    # 这里考虑了OWL中定义的对象属性和数据属性
    properties = set(g.subjects(RDF.type, OWL.ObjectProperty)) | set(g.subjects(RDF.type, OWL.DatatypeProperty))
    num_properties = len(properties)
    eda={'number of classes':num_classes, 'number of properties':num_properties}
    return eda
 


if __name__=="__main__":
    # read locl agrovc rdf 
    
    #agrovc = rdflib.Graph()
    #agrovc.parse(r"c:\data\KG\agrovoc_2023_core.rdf")
    file_path=r"C:\data\KG\C3PO\onto-elzeard-c3po-cropManagement-module.owl"
    out_path=r"C:\data\KG\C3PO\onto-elzeard-c3po-cropManagement-module.ttl"
    g=rdflib.Graph()
    g.parse(file_path)
    g.serialize(destination=out_path,format='turtle')#将其转换为ttl方便查看
 
    # Loop through each triple in the graph (subj, pred, obj)
    for subj, pred, obj in g:
        # Check if there is at least one triple in the Graph
        if (subj, pred, obj) not in g:
            raise Exception("It better be!")

    # Print the number of "triples" in the Graph
    print(f"Graph g has {len(g)} statements.")
    # Prints: Graph g has 86 statements.

    # Print out the entire Graph in the RDF Turtle format
    #print(plant.serialize(format="turtle"))
    eda=edaRdf(g)
    print(eda)
 

Graph g has 1564 statements.
{'number of classes': 172, 'number of properties': 121}


对本体进行查询，返回对应的值
1. 由于感兴趣的是agriculture process相关的内容，故查询label或comment中包含该语句的三元组
2. 基于得到的三元组，进一步返回其其对应的值，包括上下文，comment，以及其他属性


In [26]:

# 定义SPARQL查询
# 此查询查找rdfs:comment或rdfs:label属性中包含"process"的三元组
sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?s 
WHERE {
    ?s  ?predicate ?object .
    FILTER(?predicate = rdfs:label || ?predicate = rdfs:comment) .
    FILTER(CONTAINS(LCASE(STR(?object)), "agriculture process"))
}
"""

# 执行查询
qres = g.query(sparql_query)

# 打印查询结果
if len(qres)>0:
    for row in qres:
        print(f"Subject: {row.s }")
              #, Predicate: {row.predicate}, Object: {row.object}")
else:
    print("there is no reslut")

# 第二步：对于每个找到的subject，获取其所有的predicate和object
for row in qres:
    subject = row.s 
    sparql_query_step2 = f"""
    SELECT ?predicate ?object
    WHERE {{
        <{subject}> ?predicate ?object .
    }}
    """
    results = g.query(sparql_query_step2)
    print(f"Subject: {subject}")
    for result in results:
        print(f"    Predicate: {result[0]}, Object: {result[1]}")



there is no reslut


实现基于prov本体构建一个虚拟的种植processing过程，主要是在其中实现推理过程，并通过api调用  
1. 首先研究prov-o本体是否支持推理过程，及是否能够实现owl
2. 以百香果为例，构建一个虚拟的种植过程，并使用owl实现推理过程，并使用api调用

--
调查结果：
prov-o 本体支持owl，即支持owl 推理
[prov-o](https://www.w3.org/TR/prov-o/)

In [1]:
 import rdflib

# 创建一个图对象
g = rdflib.Graph()

# 加载PROV-O本体
prov_o_url = "https://www.w3.org/ns/prov-o"
g.parse(prov_o_url)

# 现在本体已经加载到图g中，您可以进行进一步的操作，比如查询等
print(f"图中包含的三元组数量: {len(g)}")


图中包含的三元组数量: 1146


考虑到owlready2对本体的创建，以及与python的面向对象方法结合更好，尝试使用owlready2来构建基本的类，并验证推理

An ontology has the following attributes:
本体具有以下属性：

.base_iri : base IRI for the ontology
 
.imported_ontologies : the list of imported ontologies (see below)

---
1. 首先创建一个本体，命名为agritom 
2. 导入prov本体，从而使得agritom本体具有prov的所有属性，并在次基础上，扩展对应的类与属性
3. 建立本体中与现有词表agrovoc的对应关系skos:match 对应的概念concept

查询agrovc中与activities对应的concept，并返回label，comment等信息,但由于agrovoc数据过大，故基于SPARQLWrapper查询sqparl point数据

进一步定义批次操作，
在本身的本体中不建立相关的农产品概念网络，而是

定义农产品类，其指向某个Concept，通过这个Concpt说明其具体的知识背景

定义批次类，其也是prov-o中activity，但其包含了多个农事活动。

在PROV-O（W3C的Provenance Ontology）中，prov:Activity用于表示在某个时间区间内发生的事情或过程。虽然PROV-O本身不直接提供一种机制来表示活动（Activity）之间的包含（或层次）关系，您可以通过一些方法来模拟这种关系。
---

[agrovoc paper](https://www.semantic-web-journal.net/system/files/swj274_1.pdf)

----
在本体处理本体基础上，结合graphdb数据库，通过http，将数据写入数据库中，并进一步实现数据的读取。

In [ ]:
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple owlready2
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple SPARQLWrapper

总结而言，使用owlready2比较适合用于初始的owl形式的定义，因为其可以比较好的检查owl语法，但其bug与限制太多，故还应该结合rdflib进行进一步的扩展定义。

如何结合大模型与本体语义，是否考虑到
1. 将三元组抽取工作交给大模型进行处理
2. 本体提供一个框架，即三元组抽取的目标框架
3. 得到的结果对应到三元组中，使得得到的数据规范化
4. 规范化的数据实现后期的推理等要求精度比较高的操作，以及更好的与业务系统整合

---

基于以上思想，那么本体设计时，将更为抽象，以增加灵活性，从而更好地适应抽取数据的需要，但是也要同时考虑业务系统的数据规范性。

具体到农业中，构建抽象prov时，就直接到批次-农事一级，而具体的农事操作，可以视为实例，

1. 结合业务系统，对应的本体逻辑为，一个经营主体，对应多个基地，一个基地与具体的地块，水塘等对应；

>经营主体的本体对应org本体

2. 在基地上，可以开展多个批次，每一个批次对应对应从农产品种植到上市的整个过程，每一个批次对应一种农产品的一个周期活动

>批次可以对应scheam中的本体

3. 农产品周期包括了农产品从一个生物周期活动，例如从育苗，到收割，到上市，但不涉及经营阶段；

>生物周期活动本体可以对应plant本体cpo3本体等与生物农业领域相关的本体

一个农产品可能跨越多个周期，比如果树，其可能经历多次结果周期，而玉米只有一次周期

4. 当农产品生物周期结束（或者预定结束）后，进入市场阶段，包括运输、冷藏、交易、品牌化等。但这个阶段依然与批次对应，直到最终消费完成。

> 考虑scheam，goodrelation本体

可以发现，以上涉及了很多概念，而这些概念的定义非常重要，因为不同的公司、地方对概念的理解不同，同时也还要考虑到未来国外销售的问题

这些概念的界定，最简单以及标准化的做法是与agrovc词表对应，但是实验发现，agrovc词表中缺少很多词汇，反而是百度上有较为详细的解释。

同时由于我们的最终目的是与供应链打通，还必须考虑与gpt，iso等标准的融合，才能实现标准化的市场对接

例如:

* [农产品购销基本信息描述 总则 标准号：GB/T 31738-2015](https://openstd.samr.gov.cn/bzgk/gb/newGbInfo?hcno=961936949F008807300FEED1D8C22B3E)

----

在以上基本schema框架下，定义基本schema，并提交chatgpt，再载入数据，然后实现目标nlp的抽取与填充。

为了节约时间，可以考虑先定义然后在继承标准化词表本体。

----

农事同样可以看成事件（event），之前我们定义时，可以考虑event-kg以及prov-o，scheam中event的结合。才能更好地体现农产品的溯源特征

农事活动可以认为同时继承了prov:Activity,sem:Event, scheam:Event

而批次 batch继承了schema:EventSeries

同时，不论是农事，还是batch其概念对应都应该与相关的概念对应，例如agrovc或c3po中



In [1]:

from owlready2 import *
import rdflib#here is used to search the remote owl
#from SPARQLWrapper import SPARQLWrapper, JSON#here is used to search the remote owl
from owlready2 import default_world
from SPARQLWrapper import SPARQLWrapper, JSON
def owl_description(owl):
    """

    Parameters:owl :输入本体
    Returns:本体的描述
    """
    class_len=len(list(owl.classes()))#类的数目
    indi_len=len(list(owl.individuals() ))
    obj_len=len(list(owl.object_properties()))#对象属性的数目
    data_len=len(list(owl.data_properties()))#数据属性的数目
    meta_comment=owl.metadata.comment
 
    return "本体共有"+str(class_len)+"类,"+str(indi_len)+"个个体,"+str(obj_len)+"个对象属性,"+str(data_len)+"个数据属性"+str(meta_comment)

def sqarql_Json(query,endpoint):
    """_实现输入query后返回json文件以及描述，对应sparql endpoint_

    Args:
        query (_type_): _description_
        endpoint (_type_): _description_

    Returns:
        _type_: _description_
    """
 
    endpoint_url = endpoint
    query =query
  # 构造SPARQL查询
  # 使用SPARQLWrapper执行查询
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #for result in results["results"]["bindings"]:
        #label = result['label']['value'] if 'label' in result else 'No label found'
        #definition = result['definition']['value'] if 'definition' in result else 'No definition found'
        #print(f"Label: {label}")
       # print(f"Definition: {definition}\n")
  # 打印结果
    return results


if __name__=="__main__":
    agri_tom=get_ontology("https:www.whu.bdi.tom/agri_tom.owl")#创建本地的owl，但没有文件化
    agri_tom.metadata.comment.append("This is the owl file about the agriclture data and resoning")
    #agrivoc=get_ontology("https://aims.fao.org/aos/agrontology").load()#农业词汇本体,远程下载可能有问题
    prov_onto=get_ontology("https://www.w3.org/ns/prov-o").load()#溯源活动本体，远端
    skos_onto = get_ontology("http://www.w3.org/2004/02/skos/core#").load()#导入远端skos本体
    sem=get_ontology("http://semanticweb.cs.vu.nl/2009/11/sem/")
    #agri_onto=get_ontology("http://aims.fao.org/aos/agrovoc/").load()
    # 打印prov-o本体的描述
   
    print('prov-o 本体：',owl_description(prov_onto))
    print('skos本体',owl_description(skos_onto))
    print(f'prov 本体中类:{list(prov_onto.classes())}')
    print(f'skos 本体中类:{list(skos_onto.classes())}')
    # 打印Activity类的信息
    prov_activity = prov_onto.search_one(iri="*Activity")#无法直接通过prov_onto.Activity得到Activity类，只能使用iri进行查询
    skos_concept=skos_onto["Concept"]#skos中类可以直接通过类名访问
    if prov_activity:
         
        print("\nprov.Activity类的信息：")
        print(f"IRI: {prov_activity.iri}")
        print(f"标签: {prov_activity.label}")
        print(f"注释: {prov_activity.comment}")
    else:
        print("\nprov-o本体中未找到Activity类。")
    #实现创建的agri_tom本体导入prov，skos本体从而使用prov，skos中类于属性
    agri_tom.imported_ontologies.append(prov_onto)
    agri_tom.imported_ontologies.append(skos_onto)
    agri_tom.imported_ontologies.append(sem)
    print('agri_tom 本体：',owl_description(agri_tom))
    #agri_tom.save(file = r"C:\data\KG\agri_tom.oewl", format = "rdfxml")
    # as import the prov-o owl into my agri_tom owl, then I can use the prov class and properties to create subclasses 
    with agri_tom:
 
        class Batch():
            pass
        Batch.label.append('[zh]')
        Batch.label.append('Agriculture Batch[en]')


        class AgriProcess(prov_activity):
            pass
        AgriProcess.label.append('农业周期[zh]')
        AgriProcess.label.append('Agriculture Process[en]')
                AgriProcess.comment='农业批次包含了从种植到上市销售的整个周期，注意一个批次对应一种农产品，一个地块'
        
        #class CultivateProcess(AgriProcess):
           # pass
       # CultivateProcess.label.append('翻土农事[zh]')
        #CultivateProcess.label.append('Cultivate Process[en]')
       # CultivateProcess.comment='农事中的翻土'

       # class SeedingProcess(AgriProcess):
         #   pass
        #SeedingProcess.label.append('播种农事[zh]')
        #SeedingProcess.label.append('Seeding Process[en]')
        #SeedingProcess.comment='农事中的播种'
        #实例化本体
            # 同样地，创建一个属性表示与skos.Concept的关系
        class isKindOfSkosConcept(ObjectProperty, FunctionalProperty):
            domain = [prov_activity]
            range = [skos_onto.Concept]
    #activ1=CultivateProcess('耕地翻土2011')
    #activ2=SeedingProcess('播种2011')
    #activ2.wasStartedBy=[activ1]# 实例本体间关系建立
    #print(activ2.wasStartedBy)


    #agri_tom.save(file=r"C:\data\KG\AGRI_TOM\agri_tom.OWL")
    #查询远端农业词表中的对应概念
    query= """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?label ?definition WHERE {
        <http://aims.fao.org/aos/agrovoc/c_330834> skos:prefLabel ?label ;
                                                    skos:definition ?definition .
        FILTER (LANG(?label) = "en")
    }
    """
    endpoint="http://agrovoc.fao.org/sparql"
    rs=sqarql_Json(query, endpoint)

    #实现owlready2中owl向rdflib中graph的转换，不需要通过中介文件
    # 使用 owlready2 本体的 world.graph
    # 这里我们使用 default_world.graph 因为 agri_tom 本体是在默认世界中创建的
    graph_data = default_world.as_rdflib_graph().serialize(format='turtle')
    graph = rdflib.Graph()
    graph.parse(data=graph_data, format='turtle')

    # 3. 在 rdflib 图中添加命名空间并定义一个实例

    # schema，even-kg直接使用rdflib进行命名空间载入
    ASGV = rdflib.Namespace("https://aims.fao.org/aos/agrovoc/")
    graph.bind("asgv", ASGV)
    bdi=rdflib.Namespace("https:www.whu.bdi.tom/agri_tom.owl#")
    graph.bind("bdi",bdi)
    sem=rdflib.Namespace("http://semanticweb.cs.vu.nl/2009/11/sem/")
    graph.bind("sem",sem)
    # 定义一个特定的实例
    concept = rdflib.URIRef("http://aims.fao.org/aos/agrovoc/c_330834")
    graph.add((concept, rdflib.RDF.type, ASGV.Concept))
    for stmt in graph:
        print(stmt)
    graph.serialize(destination=r'C:\data\KG\AGRI_TOM\agri_tom.ttl', format="turtle")
    


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#specializationOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov.alternateOf]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#wasRevisionOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov.wasDerivedFrom]; I'm trying to fix it...


prov-o 本体： 本体共有31类,0个个体,44个对象属性,6个数据属性[]
skos本体 本体共有4类,0个个体,17个对象属性,1个数据属性[]
prov 本体中类:[owl.Thing, prov.Revision, prov.Agent, prov.Delegation, prov.Activity, prov.ActivityInfluence, prov.AgentInfluence, prov.Entity, prov.Location, prov.InstantaneousEvent, prov.EntityInfluence, prov.Generation, prov.Influence, prov.Derivation, prov.End, prov.Start, prov.Collection, prov.Association, prov.Plan, prov.PrimarySource, prov.Role, prov.Usage, prov.Invalidation, prov.Attribution, prov.Communication, prov.Quotation, prov.Bundle, prov.EmptyCollection, prov.Organization, prov.Person, prov.SoftwareAgent]
skos 本体中类:[core.Concept, core.ConceptScheme, core.Collection, core.OrderedCollection]

prov.Activity类的信息：
IRI: http://www.w3.org/ns/prov#Activity
标签: ['Activity']
注释: []
agri_tom 本体： 本体共有0类,0个个体,0个对象属性,0个数据属性['This is the owl file about the agriclture data and resoning']


NameError: name 'AgriProcess' is not defined

以上实现了基本的农事本体，在此基础上根据网络中文本，进行三元组抽取，然后得到相关的农事知识图谱，并依据农事知识图谱进行农事知识问答。
1. 数据来源 [桃树栽培管理农事月历](https://www.sohu.com/a/320979381_120044303)
2. 创建农产品桃，并于agrovc中对应概念对应，同时创建子概念艳红桃，并可以考虑对应不同的数据源包括
   * [百度百科](https://baike.baidu.com/item/%E8%89%B3%E7%BA%A2%E6%A1%83/3554768)
3. 由于返回的文本很多与具体的产品不一定吻合，故通过word2vec判断中文的相似性，找到最吻合的农产品概念

In [1]:
import pandas as pd
from gensim.models import Word2Vec, KeyedVectors
# 将SPARQL查询结果的JSON转换为DataFrame
def json_to_dataframe(json_results):
    data = [
        {
            "Concept URL": result["concept"]["value"],
            "中文标签": result.get("label_zh", {}).get("value", None),
            "英文标签": result.get("label_en", {}).get("value", None),
            "定义": result.get("definition", {}).get("value", None),
            "别名": result.get("altLabel", {}).get("value", None)
        }
        for result in json_results["results"]["bindings"]
    ]
    df = pd.DataFrame(data)
    return df
query="""
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT ?concept ?label_zh ?label_en ?definition ?altLabel
WHERE {
  ?concept skos:prefLabel ?label_zh .
  OPTIONAL { ?concept skos:prefLabel ?label_en FILTER (LANGMATCHES(LANG(?label_en), "en")) }
  OPTIONAL { ?concept skos:definition ?definition FILTER (LANGMATCHES(LANG(?definition), "zh")) }
  OPTIONAL { ?concept skos:altLabel ?altLabel FILTER (LANGMATCHES(LANG(?altLabel), "zh")) }
  FILTER (LANGMATCHES(LANG(?label_zh), "zh"))
  FILTER CONTAINS(?label_zh, "周期")
}
"""
endpoint="http://agrovoc.fao.org/sparql"
rs=sqarql_Json(query, endpoint)
df = json_to_dataframe(rs)
print(df)

NameError: name 'sqarql_Json' is not defined

In [32]:
import numpy as np
import pandas as pd
import jieba

# 加载词向量
file_path = r'C:\data\NLP\tencent-ailab-embedding-zh-d200-v0.2.0-s.txt'
word_vectors = {}
with open(file_path, 'r', encoding='utf-8') as f:
    next(f)  # 跳过第一行
    for line in f:
        word, vector_str = line.split(' ', 1)
        vector = np.array([float(x) for x in vector_str.split()])
        word_vectors[word] = vector

words = list(word_vectors.keys())
word_to_id = {w: i for i, w in enumerate(words)}
id_to_word = {i: w for i, w in enumerate(words)}
word_vectors_matrix = np.stack(list(word_vectors.values()))

# 获取短语向量
def get_vector_for_phrase(phrase, word_vectors):
    words = jieba.lcut(phrase)
    print(words)
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        print("没有找到词向量。")
        return None

 
 

# 找到最相似的本体
def find_most_similar_concept(target_vector, word_vectors_matrix, df):
    norms = np.linalg.norm(word_vectors_matrix, axis=1) * np.linalg.norm(target_vector)
    similarity = np.dot(word_vectors_matrix, target_vector) / norms
    max_index = np.argmax(similarity)
    most_similar_word = id_to_word[max_index]
    similar_concept_row = df[df['中文标签'].str.contains(most_similar_word, na=False)]
    if not similar_concept_row.empty:
        return similar_concept_row.iloc[0]
    else:
        return None

# 获取目标短语的向量
target_vector = get_vector_for_phrase("油桃", word_vectors)
if target_vector is not None:
    most_similar_concept_info = find_most_similar_concept(target_vector, word_vectors_matrix, df)
    if most_similar_concept_info is not None:
        print("找到的最相似的本体信息:")
        print(most_similar_concept_info)
    else:
        print("没有找到相似的本体。")
else:
    print("目标短语的所有词都不在词向量模型中。")


['油桃']
找到的最相似的本体信息:
Concept URL    http://aims.fao.org/aos/agrovoc/c_5100
中文标签                                               油桃
英文标签                                       nectarines
定义                                               None
别名                                               None
Name: 4, dtype: object


In [22]:
pip install --upgrade gensim

Note: you may need to restart the kernel to use updated packages.


本例中，‘桃’有直接的概念[桃](http://aims.fao.org/aos/agrovoc/c_5638)，而‘红艳桃’为该概念的子概念，建立其关系

通过rdflib可以比较简单的实现以上目标，具体的步骤如下：

1. 基于rdflib
2. 读取本地ttl r'C:\data\KG\AGRI_TOM\agri_tom.ttl
3. 返回agri_tom.ttl中已有的命名空间，并使用其中bdi对应命名空间作为新概念的前缀
4. 创建一个skos:Concpet,其label是'艳红桃'
5. 建立该concept与agrovc中http://aims.fao.org/aos/agrovoc/c_5638 的关系，新的概念是其子类

In [17]:
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, SKOS

# 1. 读取本地Turtle文件
graph = rdflib.Graph()
graph.parse(r'C:\data\KG\AGRI_TOM\agri_tom.ttl', format='turtle')

# 2. 返回文件中已有的命名空间，并寻找与bdi对应的命名空间
for prefix, namespace in graph.namespaces():
    if prefix == "bdi":
        print(namespace)
        bdi_namespace = Namespace(namespace)
        break

# 2. 创建skos:Concept实例
#new_concept = URIRef("http://example.org/concept/yanhongtao")
new_concept=bdi_namespace["yanhongtao"]
label = Literal("艳红桃", lang="zh")

# 定义SKOS命名空间
#skos = Namespace("http://www.w3.org/2004/02/skos/core#")

# 添加新概念到图中
graph.add((new_concept, RDF.type, SKOS.Concept))
graph.add((new_concept, SKOS.prefLabel, label))

# 3. 建立新概念与Agrovoc中指定概念的子类关系
agrovoc_concept = URIRef("http://aims.fao.org/aos/agrovoc/c_5638")

# 在SKOS中，'broader'用于表示一个概念是另一个概念的子类（更具体的概念）
graph.add((new_concept, SKOS.broader, agrovoc_concept))

# 保存更改回本地文件（如果需要）
graph.serialize(destination=r'C:\data\KG\AGRI_TOM\agri_tom.ttl', format='turtle')

print("Completed adding new concept and its relationship.")

https:www.whu.bdi.tom/agri_tom.owl#
Completed adding new concept and its relationship.


In [11]:
#直接通过类名查询prov-o中类无法得到，不知道是什么原因
activity_cls = 
print(activity_cls)skos_onto["Concept"]

core.Concept


通过agrovc sparql point的查询，确定了activies的对应类，由于agrovc是skos结构，其中activies是一个concept实例，而非类，故本文所定义的

activity应该与其是概念对应关系，

In [ ]:
import rdflib
from rdflib.namespace import RDF, SKOS, Namespace

# 创建一个新的图
g = rdflib.Graph()

# 定义命名空间
PROV = Namespace("http://www.w3.org/ns/prov#")
AGRI = Namespace("http://example.org/agri#")  # 假定的农业本体命名空间

# 创建一个活动实例和SKOS概念实例
activity = AGRI["myActivity"]
concept = rdflib.URIRef("http://aims.fao.org/aos/agrovoc/c_330834")

# 将活动实例与SKOS概念链接
g.add((activity, RDF.type, PROV.Activity))
g.add((activity, SKOS.closeMatch, concept))  # 使用SKOS的匹配属性表示关系

# 序列化图查看结果
print(g.serialize(format="turtle").decode("utf-8"))

In [6]:
from owlready2 import *

# 步骤1: 访问prov-o本体
prov_o_iri = "http://www.w3.org/ns/prov-o"
prov_onto = get_ontology(prov_o_iri).load()

# 步骤4: 返回prov-o中存在的类
prov_classes = list(prov_onto.classes())
print("Prov-O Classes:", prov_classes)

# 步骤5: 访问prov-o中的类，举例访问Entity类
entity_class = prov_onto.Entity
print("Entity Class in Prov-O:", entity_class)

Prov-O Classes: [owl.Thing, prov.Revision, prov.Agent, prov.Delegation, prov.Activity, prov.ActivityInfluence, prov.AgentInfluence, prov.Entity, prov.Location, prov.InstantaneousEvent, prov.EntityInfluence, prov.Generation, prov.Influence, prov.Derivation, prov.End, prov.Start, prov.Collection, prov.Association, prov.Plan, prov.PrimarySource, prov.Role, prov.Usage, prov.Invalidation, prov.Attribution, prov.Communication, prov.Quotation, prov.Bundle, prov.EmptyCollection, prov.Organization, prov.Person, prov.SoftwareAgent]
Entity Class in Prov-O: None


In [1]:
from rdflib import URIRef

# 确保使用正确的命名空间前缀
# 对于prov-o本体中的类，我们可以这样访问它们
entity_class = prov_onto.search_one(iri="*Entity")

# 如果上述方法不起作用，我们可以尝试直接使用URI访问
# 这是一种更直接的方法，但通常不是首选
entity_class_direct = URIRef("http://www.w3.org/ns/prov#Entity")

print("Entity Class in Prov-O using search_one:", entity_class)
print("Direct URIRef access to Entity Class in Prov-O:", entity_class_direct)


NameError: name 'prov_onto' is not defined

In [16]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple deeponto

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/89.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/89.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/89.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/89.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/89.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/89.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/89.7 MB 81.9 kB/s eta 0:18:16
     ---------------------------------------- 0.0/89.7 MB 81.9 kB/s eta 0:18:16
     ---------------------------------------- 0.0/89.7 MB 93.7 kB/s eta 0:15:58
     --------------------------------------- 0.1/89.7 MB 136.5 kB/s eta 0:10:57
     --------------------------------------- 0.1/89.7 MB 140.3 kB/s eta 0:10:40
     --------------------------------------- 0.1/89.7 MB 140.3 kB/s eta 0:10:40
     ----------------------

In [ ]:
import deeponto

In [1]:
from owlready2 import *

# 第1步：创建本体并命名为agritom
onto = get_ontology("http://example.org/agritom.owl")

with onto:
    # 第2步：定义需要导入的本体。注意，实际操作中需要替换为正确的prov和skos本体URL
    # 由于示例中的URL可能无法直接访问，这里仅作为示例展示。请根据实际情况替换为有效的本体URL。
    prov = get_ontology("http://www.w3.org/ns/prov#").load()
    skos = get_ontology("http://www.w3.org/2004/02/skos/core#").load()

    # 导入prov和skos本体
    import_ontologies([prov, skos])

    # 第3步：定义agri_active类，继承了prov的Activity类以及skos的Concept类
    class agri_active(prov.Activity, skos.Concept):
        pass

    # 第4步：建立与现有词表agrovoc的对应关系
    # 注意：由于OWLReady2中没有直接支持skos:match属性，这里我们添加一个注释来示意这个步骤。
    # 在实际应用中，您可能需要以不同的方式来实现这一关系，例如通过RDFLib直接操作RDF图。
    agri_active.comment.append("This class is matched with the agrovoc concept http://aims.fao.org/aos/agrovoc/c_330834")

# 保存本体到文件
onto.save(file="agritom.owl", format="rdfxml")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


ValueError: Cannot create a new ontology inside a with onto:... block!